# Bloodrager
Leshy Bloodrager barbarian, critfisher build

**Feats:** **1.** Extended Reach, **2.** Bloodrager Dedication, **4.** Rising Blood Magic, **6.** Siphon Magic, **10.** Hematocritical, **12.** Surging Blood Magic, **18.** Exultant Blood Magic

**Spells**: **cantrips** ignition or live wire, electric arc; **1st** Sure Strike; **2nd** Brine Dragon Bile; **3rd** Haste or Blazing Bolt or Breathe Fire or Fireball or Organsight (signature)

**Equipment:** Rooting Flaming Greatpick or Greatsword, (Greater) Phantasmal Doorknob

In [ ]:
# Install in jupyterlite
%pip install -q pathfinder2e-stats

import xarray

import pathfinder2e_stats as pf2

In [ ]:
level = 14

spell_slot_rank = (
    pf2.level2rank(level, dedication=True) - 1
)  # max - 1, recoverable with Siphon Magic
use_hematocritical = level >= 10
use_rooting_rune = level >= 7  # clumsy 1 on a crit
use_flaming_rune = level >= 10
use_sword = False  # off-guard to ranged spells on a crit
use_greater_phantasmal_doorknob = level >= 10  # off-guard to ranged spells on a crit

## Attack bonus progrssion
Weapon vs. spell vs. Organsight Medicine checks

In [ ]:
atk_bonus = xarray.Dataset(
    {
        "weapon": pf2.tables.SIMPLE_PC.weapon_attack_bonus.barbarian.sum("component"),
        "spell": pf2.tables.SIMPLE_PC.spell_attack_bonus.barbarian.sum("component"),
        "organsight": (
            pf2.tables.PC.ability_bonus.boosts.sel(initial=1, drop=True)
            + pf2.tables.PC.skill_proficiency.others.sel(priority=1, drop=True)
            + pf2.tables.PC.skill_item_bonus.medicine
            + pf2.tables.PC.level
            + 2  # Circumstance
        ),
    }
)
atk_bonus.to_pandas()

Let's select 3 standard targets:

- level -2 henchman, all defenses are low
- at-level monster, all defenses are moderate
- level +2 boss, all defenses are high

In [ ]:
rank = pf2.level2rank(level)
defenses = pf2.tables.SIMPLE_NPC[["AC", "saving_throws", "recall_knowledge"]].sel(
    level=level
)
AC = defenses.AC
saves = defenses.saving_throws
defenses.to_array("kind").to_pandas()

## Build damage profiles for weapon and spells

In [ ]:
STR = (
    pf2.tables.PC.ability_bonus.boosts.sel(initial=4) + pf2.tables.PC.ability_bonus.apex
)
weapon_specialization = pf2.tables.PC.weapon_specialization.barbarian
rage_weapon = pf2.tables.PC.rage.bloodrager_weapon
weapon_dmg_bonus = (STR + weapon_specialization + rage_weapon).sel(level=level).item()
rage_bleed = pf2.tables.PC.rage.bloodrager_bleed.sel(level=level).item()
weapon_dice = pf2.tables.PC.weapon_dice.striking_rune.sel(level=level).item()

if use_sword:
    # Greatsword with extended reach
    weapon = pf2.armory.pathfinder.melee.greatsword(
        weapon_dice, weapon_dmg_bonus
    ).reduce_die()
else:
    # Greatpick with extended reach
    weapon = pf2.armory.pathfinder.melee.greatpick(
        weapon_dice, weapon_dmg_bonus
    ).reduce_die()
    if level >= 5:
        weapon += pf2.armory.critical_specialization.pick(weapon_dice)

if use_flaming_rune:
    weapon += pf2.armory.runes.flaming()

weapon += pf2.Damage("bleed", 0, 0, rage_bleed, persistent=True)
weapon

In [ ]:
def rage_spell(
    level: int, type_: str, *, persistent: bool = False, drained: int = 2
) -> dict[pf2.DoS, list[pf2.Damage]]:
    raw = pf2.tables.PC.rage.bloodrager_spells.sel(level=level, drained=drained).item()
    d = pf2.Damage(type_, 0, 0, raw, persistent=persistent)
    return {
        pf2.DoS.critical_success: [d.copy(multiplier=2)],
        pf2.DoS.success: [d],
        pf2.DoS.failure: [d],
    }


ignition_melee = pf2.armory.cantrips.ignition(rank, melee=True) + rage_spell(
    level, "fire"
)
ignition_melee

In [ ]:
ignition_ranged = pf2.armory.cantrips.ignition(rank, melee=False) + rage_spell(
    level, "fire"
)
ignition_ranged

In [ ]:
live_wire = pf2.armory.cantrips.live_wire(rank) + rage_spell(level, "electricity")
live_wire

In [ ]:
electric_arc = pf2.armory.cantrips.electric_arc(rank)
electric_arc

In [ ]:
breathe_fire = pf2.armory.spells.breathe_fire(spell_slot_rank)
breathe_fire

In [ ]:
brine_dragon_bile = pf2.armory.spells.brine_dragon_bile(spell_slot_rank) + rage_spell(
    level, "acid", persistent=True
)
brine_dragon_bile

In [ ]:
blazing_bolt_1action = pf2.armory.spells.blazing_bolt(
    spell_slot_rank, actions=1
) + rage_spell(level, "fire")
blazing_bolt_1action

In [ ]:
blazing_bolt_3actions = pf2.armory.spells.blazing_bolt(
    spell_slot_rank, actions=3
) + rage_spell(level, "fire")
blazing_bolt_3actions

In [ ]:
organsight = pf2.armory.spells.organsight(spell_slot_rank)
organsight

## Attack routine
- Strike (with flank) -> Hematocritical if crit -> spell, _or_
- (if hasted) Sure Strike -> Strike (with flank) -> Hematocritical -> spell

Spell is one of:
- Ignition (melee with flank)
- Ignition (ranged due to reach)
- Live Wire
- Electric Arc (1-2 targets)
- Breathe Fire / Fireball
- Blazing Bolt (1-2-3 actions)
- (out of round) Brine Dragon Bile

Spells from slots are at maximum rank -1, so that they can be cycled with Syphon Magic.

In [ ]:
sure_strike = xarray.DataArray(
    [False, True, False],
    dims=["Sure Strike"],
    coords={"Sure Strike": ["Normal", "Sure Strike", "Only on melee crit"]},
)

- Dimensions ``challenge`` and ``sure strike`` represent a what-if analysis.
  Roll dice only once and compare the results against different situations.
- For AoE spells with a saving throw (Electric Arc, Breathe Fire) roll damage
  only once, but roll saving throw individually for every target.
- For Blazing Bolt, roll both attack and damage individually for every target.

In [ ]:
pf2.set_config(
    check_independent_dims=["AoE_target", "BB_target"],
    check_dependent_dims=["challenge", "Sure Strike"],
    damage_independent_dims=["BB_target"],
    damage_dependent_dims=["challenge", "Sure Strike", "AoE_target"],
)

In [ ]:
strike = pf2.damage(
    pf2.check(
        bonus=atk_bonus.weapon.sel(level=level).item(),
        DC=AC - 2,
        fortune=sure_strike,
    ),
    weapon,
)

## What are the chances of a critical hit on the initial weapon strike?

In [ ]:
melee_crit = strike.outcome == pf2.DoS.critical_success
melee_crit.loc[{"Sure Strike": "Only on melee crit"}] = True
melee_crit.mean("roll").round(3).to_pandas() * 100.0

The conditions of the next spell change depending on the strike and equipment:

- If the strike was critical, we can use Hematocritical
- If the weapon was rooting, the target is now Clumsy 1
- If the weapon was a sword, th target is now off-guard even if not flanked

In [ ]:
hematocritical = melee_crit if use_hematocritical else xarray.DataArray(False)
clumsy = melee_crit if use_rooting_rune else xarray.DataArray(0)
ranged_off_guard = (
    2 * melee_crit
    if (use_sword or use_greater_phantasmal_doorknob)
    else xarray.DataArray(0)
)

## Roll damage for the spells

In [ ]:
ignition_melee_dmg = pf2.damage(
    pf2.check(
        atk_bonus.spell.sel(level=level) - 5,
        DC=AC - 2 - clumsy,
        fortune=hematocritical,
    ),
    ignition_melee,
)

ignition_ranged_dmg = pf2.damage(
    pf2.check(
        atk_bonus.spell.sel(level=level) - 5,
        DC=AC - clumsy - ranged_off_guard,
        fortune=hematocritical,
    ),
    ignition_ranged,
)

live_wire_dmg = pf2.damage(
    pf2.check(
        atk_bonus.spell.sel(level=level) - 5,
        DC=AC - clumsy - ranged_off_guard,
        fortune=hematocritical,
    ),
    live_wire,
)

AoE_target = xarray.DataArray(
    [1, 0, 0],
    dims=["AoE_target"],
    coords={"AoE_target": ["Strike target", "target 2", "target 3"]},
)

electric_arc_dmg = pf2.damage(
    pf2.check(
        saves - AoE_target[:2] * clumsy,
        DC=atk_bonus.spell.sel(level=level) + 10,
        misfortune=hematocritical,
    ),
    electric_arc,
).rename({"AoE_target": "target"})  # Align with Blazing Bolt target later

breathe_fire_dmg = pf2.damage(
    pf2.check(
        saves - AoE_target * clumsy,
        DC=atk_bonus.spell.sel(level=level) + 10,
        misfortune=hematocritical,
    ),
    breathe_fire,
).rename({"AoE_target": "target"})

We need to use a different dimension from before because
the above AoEs have dependent damage rolls (roll only once for all targets),
whereas Blazing Bolt is independent (roll separately for each target).
See ``set_config`` above.

In [ ]:
bb_target = AoE_target.rename({"AoE_target": "BB_target"})

blazing_bolt_check = pf2.check(
    atk_bonus.spell.sel(level=level) - 5,
    DC=AC - (clumsy + ranged_off_guard) * bb_target,
    fortune=hematocritical,
)
blazing_bolt_1action_dmg = pf2.damage(
    blazing_bolt_check,
    blazing_bolt_1action,
).isel(BB_target=0, drop=True)

blazing_bolt_23actions_dmg = pf2.damage(
    blazing_bolt_check,
    blazing_bolt_3actions,
).rename({"BB_target": "target"})

Also show:

- A second iterative strike
- a standalone 3-actions Blazing Bolt
- an out-of-round Brine Dragon Bile
- additional damage from Organsight, applied to the initial and iterative Strike on each round

In [ ]:
strike2 = pf2.damage(
    pf2.check(
        bonus=atk_bonus.spell.sel(level=level).item() - 5,
        DC=AC - 2 - clumsy,
    ),
    weapon,
)

blazing_bolt_23actions_noMAP_dmg = pf2.damage(
    pf2.check(
        atk_bonus.spell.sel(level=level),
        DC=AC,
        independent_dims={"BB_target": 3},
    ),
    blazing_bolt_3actions,
).rename({"BB_target": "target"})

brine_dragon_bile_dmg = pf2.damage(
    pf2.check(
        atk_bonus.spell.sel(level=level),
        DC=AC,
    ),
    brine_dragon_bile,
)

In [ ]:
organsight_check = pf2.check(
    atk_bonus.organsight.sel(level=level),
    DC=defenses.recall_knowledge,
)

organsight_check["recall_knowledge_outcome"] = organsight_check.outcome
organsight_check["strike_outcome"] = xarray.concat(
    [
        strike.outcome,
        xarray.where(
            strike.outcome >= pf2.DoS.success,
            pf2.DoS.no_roll,
            strike2.outcome,
        ),
    ],
    dim="strike",
)
organsight_check["outcome"] = xarray.where(
    organsight_check.outcome >= pf2.DoS.success,
    xarray.where(
        organsight_check.strike_outcome >= pf2.DoS.success,
        organsight_check.strike_outcome,
        pf2.DoS.no_roll,
    ),
    pf2.DoS.no_roll,
)
organsight_check.coords["strike"] = ["initial", "iterative"]
# "Only on melee crit" makes no sense here
organsight_check = organsight_check.isel({"Sure Strike": slice(2)})
organsight_dmg = pf2.damage(
    organsight_check,
    organsight,
    independent_dims=["strike"],
)

## Mean damage for every action

In [ ]:
rows = {
    "Weapon Strike (flanked)": strike,
    "Iterative Weapon Strike (flanked) (MAP-5)": strike2,
    "Organsight (first strike)": organsight_dmg.isel(strike=0, drop=True),
    "Organsight (iterative strike)": organsight_dmg.isel(strike=1, drop=True),
    "Ignition (melee, flanked) (MAP-5)": ignition_melee_dmg,
    "Ignition (ranged) (MAP-5)": ignition_ranged_dmg,
    "Live Wire (MAP-5)": live_wire_dmg,
    "Electric Arc (1 target)": electric_arc_dmg.isel(target=slice(1)),
    "Electric Arc (2 targets)": electric_arc_dmg,
    "Breathe Fire (1 target)": breathe_fire_dmg.isel(target=slice(1)),
    "Breathe Fire (2 targets)": breathe_fire_dmg.isel(target=slice(2)),
    "Breathe Fire (3 targets)": breathe_fire_dmg,
    "Blazing Bolt > (MAP-5)": blazing_bolt_1action_dmg,
    "Blazing Bolt >> (MAP-5)": blazing_bolt_23actions_dmg.isel(target=slice(2)),
    "Blazing Bolt >>> (MAP-5)": blazing_bolt_23actions_dmg,
    "Blazing Bolt >>> (standalone)": blazing_bolt_23actions_noMAP_dmg,
    "Brine Dragon Bile (standalone)": brine_dragon_bile_dmg,
}

damages = []
for dmg in rows.values():
    dmg = dmg.total_damage.mean("roll")
    if "target" in dmg.dims:
        dmg = dmg.sum("target")
    damages.append(dmg)

total_damage = xarray.concat(damages, dim="activity", join="outer", coords="minimal")
total_damage.coords["activity"] = list(rows)

total_damage.loc[
    {"activity": total_damage.activity[0], "Sure Strike": "Only on melee crit"}
] = float("nan")
total_damage.loc[
    {"activity": total_damage.activity[-2:], "Sure Strike": "Only on melee crit"}
] = float("nan")
total_damage.loc[
    {"activity": total_damage.activity[-3:], "Sure Strike": "Sure Strike"}
] = float("nan")
total_damage.stack(col=["challenge", "Sure Strike"]).to_pandas().round(1)

## Outcome probability for the initial Strike

In [ ]:
(
    pf2.outcome_counts(strike)
    .isel({"Sure Strike": slice(2)})
    .stack(col=["challenge", "Sure Strike"])
    .to_pandas()
    .round(3)
    * 100.0
)

## Outcome probability for the iterative Strike

In [ ]:
(
    pf2.outcome_counts(strike2)
    .stack(col=["challenge", "Sure Strike"])
    .to_pandas()
    .round(3)
    * 100.0
)

## Outcome probability for Ignition (melee)

In [ ]:
(
    pf2.outcome_counts(ignition_melee_dmg)
    .stack(col=["challenge", "Sure Strike"])
    .to_pandas()
    .round(3)
    * 100.0
)

## Outcome probability for Electric Arc / Breathe Fire / Fireball

In [ ]:
(
    pf2.outcome_counts(electric_arc_dmg)
    .stack(row=["target", "outcome"])
    .stack(col=["challenge", "Sure Strike"])
    .to_pandas()
    .round(3)
    * 100.0
)

## Outcome probability for Organsight
### Recall Knowledege

In [ ]:
(
    pf2.outcome_counts(organsight_dmg.recall_knowledge_outcome).to_pandas().T.round(3)
    * 100.0
)

### Triggering Strike
The iterative strike is "no roll" whenever the initial strike connects, as the damage from Organsight can only be discharged once per round.

In [ ]:
(
    pf2.outcome_counts(organsight_dmg.strike_outcome)
    .stack(row=["strike", "outcome"])
    .stack(col=["challenge", "Sure Strike"])
    .to_pandas()
    .round(3)
    * 100.0
)

### Combined chance to apply Organsight

In [ ]:
(
    pf2.outcome_counts(organsight_dmg)
    .stack(row=["strike", "outcome"])
    .stack(col=["challenge", "Sure Strike"])
    .to_pandas()
    .round(3)
    * 100.0
)